This notebook was prepared by [Donne Martin](https://github.com/donnemartin). Source and license info is on [GitHub](https://github.com/donnemartin/interactive-coding-challenges).

# Challenge Notebook

## Problem: Given a list of 2x2 matrices, minimize the cost of matrix multiplication.

* [Constraints](#Constraints)
* [Test Cases](#Test-Cases)
* [Algorithm](#Algorithm)
* [Code](#Code)
* [Unit Test](#Unit-Test)
* [Solution Notebook](#Solution-Notebook)

## Constraints

* Do we just want to calculate the cost and not list the actual order of operations?
    * Yes
* Can we assume the inputs are valid?
    * No
* Can we assume this fits memory?
    * Yes

## Test Cases

* None -> Exception
* [] -> 0
* [Matrix(2, 3), Matrix(3, 6), Matrix(6, 4), Matrix(4, 5)] -> 124

## Algorithm

Refer to the [Solution Notebook]().  If you are stuck and need a hint, the solution notebook's algorithm discussion might be a good place to start.

## Code

In [1]:
class Matrix(object):

    def __init__(self, first, second):
        self.first = first
        self.second = second
        
    def __repr__(self):
        return 'M[{}, {}]'.format(self.first, self.second)
        
    def multiplies(m1, m2):
        assert m1.second == m2.first, '{}, {}'.format(m1, m2)
        return m1.first * m1.second * m2.second
        # return MatrixChain([m1, m2], m1.first * m1.second * m2.second)
    
class MatrixChain(object):
    def __init__(self, matrices, multiplies):
        assert len(matrices) > 1
        self.matrices = matrices
        self.multiplies = multiplies
        for ix in range(len(matrices) - 1):
            assert matrices[ix].second == matrices[ix + 1].first
        self.matrix = Matrix(matrices[0].first, matrices[-1].second)

    def __repr__(self):
        result = ''
        for mat in self.matrices:
            result += '{} '.format(mat)
        result += '@ {} multiplies'.format(self.multiplies)
        return result

In [2]:
class MatrixMultiplicationCost(object):

    def find_min_cost(self, matrices):
        if len(matrices) == 0:
            return 0
        # key: list of matrices
        # value: min # of multiplies to compute the product of the matrices in key, in order
        memo = {}
        
        for n_matrices in range(1, len(matrices)):
            for i in range(len(matrices) - n_matrices):
                key = tuple(range(i, i + n_matrices + 1))

                if len(key) == 2:
                    key_cost = Matrix.multiplies(matrices[key[0]], matrices[key[1]])
                    assert key not in memo
                    memo[key] = MatrixChain([matrices[key[0]], matrices[key[1]]], key_cost)
                    
                else:
                    left_term = matrices[key[0]]
                    right_term = memo[key[1:]]
                    right_term_grouped_cost = Matrix.multiplies(left_term, right_term.matrix) + right_term.multiplies
                    right_term_grouped = MatrixChain([left_term] + right_term.matrices, right_term_grouped_cost)

                    left_term = memo[key[:len(key) - 1]]
                    right_term = matrices[key[-1]]
                    left_term_grouped_cost = Matrix.multiplies(left_term.matrix, right_term) + left_term.multiplies
                    left_term_grouped = MatrixChain(left_term.matrices + [right_term], left_term_grouped_cost)
                    
                    assert key not in memo
                    if left_term_grouped.multiplies < right_term_grouped.multiplies:
                        memo[key] = left_term_grouped
                    else:
                        memo[key] = right_term_grouped
        
        return(memo[tuple(range(len(matrices)))].multiplies)
        

## Unit Test

**The following unit test is expected to fail until you solve the challenge.**

In [3]:
# %load test_find_min_cost.py
from nose.tools import assert_equal, assert_raises


class TestMatrixMultiplicationCost(object):

    def test_find_min_cost(self):
        matrix_mult_cost = MatrixMultiplicationCost()
        assert_raises(TypeError, matrix_mult_cost.find_min_cost, None)
        assert_equal(matrix_mult_cost.find_min_cost([]), 0)
        matrices = [Matrix(2, 3),
                    Matrix(3, 6),
                    Matrix(6, 4),
                    Matrix(4, 5)]
        expected_cost = 124
        assert_equal(matrix_mult_cost.find_min_cost(matrices), expected_cost)
        print('Success: test_find_min_cost')


def main():
    test = TestMatrixMultiplicationCost()
    test.test_find_min_cost()


if __name__ == '__main__':
    main()

Success: test_find_min_cost


## Solution Notebook

Review the [Solution Notebook]() for a discussion on algorithms and code solutions.